# Setup

In [ ]:
import tensorflow as tf
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Data

In [ ]:
url = 'https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/tf2.0/moore.csv'
filename = 'moores_law_data.data'
filepath = tf.keras.utils.get_file(filename, url)
target_path = '../resources/' + filename
os.system('move ' + filepath + ' ' + target_path)

In [ ]:
data = pd.read_csv('../resources/moores_law_data.data', header=None).to_numpy()
print(data)

In [ ]:
X = data[:,0].reshape(-1,1)
Y = np.log(data[:,1]) # Log transformation for linear relationship
plt.scatter(X,Y)

In [ ]:
# Partially normalise
X = X - X.mean()

# Model

In [ ]:
# Configure the wandb sweep
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.005, 0.0005]
        },
        'epochs': {
            'values': [100, 200, 500]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'rmsprop', 'adadelta', 'adagrad', 'adamax']
        },
    }
}

In [ ]:
# Initialise sweep
sweep_id = wandb.sweep(sweep_config, project='tensorflow-test', entity='kavp')

In [ ]:
# Mega function to define and train model and log results (used by the sweep)
def sweep_func():
    # Default hyperparameter values
    config_defaults = {
        'learning_rate': 0.001,
        'epochs': 100,
        'batch_size': 128,
        'optimizer': 'adam',
        'eager_mode': False,
        'loss_func': 'mse',
        'callbacks': 'lr',
        'momentum': 0.9,
    }

    # Initialise run
    wandb.init(config=config_defaults)

    # Variable to hold the sweep values
    config = wandb.config
    
    if config['eager_mode'] == True:
        tf.compat.v1.enable_eager_execution()
    elif config['eager_mode'] == False:
        tf.compat.v1.disable_eager_execution()
    else:
        raise ValueError('eager_mode property of wandb config could not be determined.') 

    # Define model
    model = tf.keras.models.Sequential([
        # Layers
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(1), # Layer for regression
    ])

    model.compile(
        optimizer=config['optimizer'],
        loss=config['loss_func'],
        run_eagerly=config['eager_mode'],
    )

    with tf.compat.v1.Session() as sess:
        r = model.fit(X, Y, validation_data=(X,Y), epochs=config['epochs'], batch_size=config['batch_size'])
        wandb.tensorflow.log(tf.compat.v1.summary.merge_all())
        wandb.log({'loss': r.history['loss'][-1], 'log_exponential_growth_factor': model.layers[0].get_weights()[0][0,0]})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['loss'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "loss"])
        wandb.log({"loss_against_epochs" : wandb.plot.line(table, "epoch", "loss", title="Training loss")})

In [ ]:
wandb.agent(sweep_id, sweep_func)

In [ ]:
wandb.finish()